In [1]:
import json

from langchain.chains.question_answering import load_qa_chain

from langchain_community.llms import Ollama
from langchain_community.embeddings.laser import LaserEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatMaritalk

from langchain_core.prompts.chat import ChatPromptTemplate

from langchain.text_splitter import RecursiveCharacterTextSplitter

# from langchain.experi

import numpy as np

import pandas as pd


- langchain                                0.3.21
- langchain-community                      0.3.20
- langchain-core                           0.3.49
- langchain-huggingface                    0.0.3
- langchain-text-splitters                 0.3.7
- maritalk                                 0.2.6
- pandas                                   2.2.3
- 

In [2]:
LaserEmbeddings.validate_environment({'lang':'por_Latn'})

2025-03-29 18:05:48,534 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2025-03-29 18:05:48,921 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2025-03-29 18:05:48,981 | INFO | laser_encoders.download_models |  - laser2.pt already downloaded
2025-03-29 18:05:48,981 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2025-03-29 18:05:48,982 | INFO | laser_encoders.download_models |  - laser2.cvocab already downloaded


{'lang': 'por_Latn',
 '_encoder_pipeline': <laser_encoders.models.LaserEncoderPipeline at 0x7f29fc2783d0>}

In [3]:
import langchain
langchain.__version__

'0.2.0'

In [4]:
# obtains the API key used to access Maritalk's API
with open('API_KEY.json', 'r') as file:
    maritalk = json.load(file)

model = ChatMaritalk(
    model='sabia-3',
    api_key=maritalk['key'],
    temperature=0.7,
    max_tokens=500,
)

In [4]:
# ollama_base_url= 'http://tempestade.facom.ufms.br:11435'
ollama_base_url = 'http://localhost:11434' 
# model = Ollama(base_url=ollama_base_url, model="phi3:medium", temperature=0)

# ollama_base_url = 'http://localhost:11434' 
model = Ollama(base_url=ollama_base_url, model="llama3.1", temperature=0)

# embeddings_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

In [5]:
embeddings_model = LaserEmbeddings(lang='por_Latn')

2025-03-29 18:05:51,854 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2025-03-29 18:05:51,946 | INFO | laser_encoders.download_models |  - laser2.pt already downloaded
2025-03-29 18:05:51,947 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2025-03-29 18:05:51,948 | INFO | laser_encoders.download_models |  - laser2.cvocab already downloaded


In [6]:
doc_folder = 'edital1'

In [7]:
documents_dir_path = f'./database/{doc_folder}'

loader = DirectoryLoader(documents_dir_path, glob='./*.pdf', loader_cls=PyPDFLoader)

loaded_pdfs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=0,
        separators=['\n', '.']
)

# text_splitter = 

pages = text_splitter.split_documents(loaded_pdfs)

In [8]:
pages

[Document(metadata={'source': 'database/edital1/edital-1.pdf', 'page': 0}, page_content='EDIT AL PR OAES/UFMS Nº 70, DE 4 DE JUNHO DE 2024.\nCAD ASTRO PARA BOLS A PERMANÊNCIA DO ME C PARA E STUD ANTE S INDÍGENAS E\nQUIL OMBOLAS NA GRADU AÇÃO - PR OGRAMA BPME C 2024\nA FUNDAÇÃO UNIVER SIDADE FEDERAL  DE MATO GROSSO DO SUL, por meio\nda Pró-Reitoria de Assun tos Estudan \x00s - Proaes, no uso de suas atribuiç ões legais, e de acordo\ncom a Portaria MEC nº 389, de 9 de maio de 2013; a Portaria Sesu/ME C nº 9, de 9 de maio de'),
 Document(metadata={'source': 'database/edital1/edital-1.pdf', 'page': 0}, page_content='2023; e a Portaria MEC nº 1.999, de 10 de novembr o de 2023, torna públic o o presen te Edital\nde cadas tro de estudan tes de graduaç ão indíg enas e quilombolas, matricula dos na UFMS,\npara solicit ação de Bolsa P ermanência do ME C - BPME C.\n \n1. DISPOSIÇ ÕES PRELIMINARE S\n1.1. Este Edital tem como obje \x00vo estabelecer as normas para o cadas tro de\nestudan tes de gra

In [9]:
vectorstore = FAISS.from_documents(
    pages,
    embeddings_model
)

vectorstore.save_local(f'database/{doc_folder}/document_index')

2025-03-29 18:06:01,375 | INFO | faiss.loader | Loading faiss with AVX2 support.
2025-03-29 18:06:01,390 | INFO | faiss.loader | Successfully loaded faiss with AVX2 support.


In [ ]:
# loads vectorstore from disk
vectorstore = FAISS.load_local(f'database/{doc_folder}/document_index', embeddings_model, allow_dangerous_deserialization=True)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={'k':20})

In [ ]:
def assistant(content: str):
    return ("assistant", content)

def user(content: str):
    return ("user", content)

In [ ]:
prompt_structure = '''
        Baseado nos seguintes documentos:
        {context}
        Responda a pergunta abaixo:
        {query}
        '''

In [ ]:
# role based prompt
qa_prompt = ChatPromptTemplate.from_messages([
        user('''Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Você sempre cita INTEGRALMENTE o item do edital que contém a resposta desejada. Se não souber a resposta, responda "Não consigo encontrar essa informação no documento". 
         Você cita seomente item necessário para resposta direta da pergunta e nada mais. Você SEMPRE cita o número do item que contém a resposta.'''),
        ('human', prompt_structure)
])

In [ ]:
# few-shot prompting
qa_prompt = ChatPromptTemplate.from_messages([
        user('''Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Você sempre cita INTEGRALMENTE o item do edital que contém a resposta desejada. Se não souber a resposta, responda "Não consigo encontrar essa informação no documento". 
         Você cita seomente item necessário para resposta direta da pergunta e nada mais. Você SEMPRE cita o número do item que contém a resposta. Aqui estão alguns exemplos:'''),
        user('''Como será a lista de espera?'''),
        assistant('''De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.'''),
        user('''Qual o número mínimo de membros das comissões temporárias constituídas pelo Conselho?'''),
        assistant('''De acordo com o Art. 61, as comissões temporárias deverão ser constituídas por, no mínimo, três membros.'''),
        user(prompt_structure)
])

In [ ]:
chain = load_qa_chain(model, chain_type='stuff', verbose=True, prompt=qa_prompt)

In [ ]:
query = input()

In [ ]:
# retrieval_result = retriver.invoke(query)
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

retrieval_result = compression_retriever.invoke(query)


In [ ]:
def reranker_retrieval(query):
    return compression_retriever.invoke(query)

def vectorstore_retrieval(query):
    retrieval_result = vectorstore.similarity_search_with_relevance_scores(query, k=10, score_threshold=0.5)
    return [page[0] for page in retrieval_result]

In [ ]:
retrieval_result = vectorstore.similarity_search_with_relevance_scores(query, k=10, score_threshold=0.5)

# retrieved context, no enrichment SOMENTE USADO NA BUSCA DIRETO PELA VECTOR STORE
retrieval_result= [page[0] for page in retrieval_result]

In [ ]:
# enriching the retrieval result PARENT DOCUMENT RETRIEVAL

# get the page index of the page with the most similar chunk
# page_idx = retrieval_result[0][0].metadata['page']

# retrieval_result = np.concatenate(
#     (
#         [x for x in pages if x.metadata['page'] == page_idx],
#         [page[0] for page in retrieval_result]),
#     axis=0
# )

In [ ]:
output = chain.invoke(
        {'input_documents':retrieval_result, 'query':query}
)

print(output['output_text'])

### Geração das respostas

In [ ]:
map_editais = {70 : 'edital1', 9 : 'edital2'}

In [ ]:
queries = pd.read_csv('tests/test_questions.csv', dtype={'numero':int})

enrichment = False

answers = []

for num_doc in [70, 9]:

    questions = queries[queries['numero'] == num_doc]['pergunta'].tolist()

    for question in questions:
        
        # query = f'De acordo com o edital {num_doc}. ' + question

        vectorstore = FAISS.load_local(f'database/{map_editais[num_doc]}/document_index', embeddings_model, allow_dangerous_deserialization=True)

        retrieval_result = reranker_retrieval(question)

        if enrichment:
            # enriching the retrieval result 

            # get the page index of the page with the most similar chunk
            page_idx = retrieval_result[0][0].metadata['page']

            retrieval_result = np.concatenate(
                (
                    [x for x in pages if x.metadata['page'] == page_idx],
                    [page[0] for page in retrieval_result]),
                axis=0
            )
        else:
            # retrieved context, no enrichment
            # retrieval_result= [page[0] for page in retrieval_result]
            pass

        output = chain.invoke(
            {'input_documents':retrieval_result, 'query':question}
        )

        answers.append(output['output_text'])

In [ ]:
result = pd.DataFrame(
    {
        'pergunta':queries['pergunta'].tolist(),
        'resposta':answers,
        'edital':queries['arquivo'].tolist()
    }
)

In [ ]:
result.to_csv(f'tests/results/{model.model}-few-shot-reranker.csv', index=False)